In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(os.listdir("../input/catboost-model/"))

# Any results you write to the current directory are saved as output.

['fork-of-treebasedapproachdata', 'competitive-data-science-predict-future-sales', 'catboost-model']
['__notebook__.ipynb', 'submission.csv', '__output__.json', 'catboost_info', '__results___files', 'custom.css', '__results__.html', 'trained_model.bin']


In [2]:
DATA_FPATH = '../input/fork-of-treebasedapproachdata/DATA.hdf'
SAMPLE_SUBMISSION_FPATH = '../input/competitive-data-science-predict-future-sales/sample_submission.csv'
TRAINED_MODEL_FPATH = '../input/catboost-model/trained_model.bin'

In [3]:
from catboost import CatBoostRegressor
regressor_kwargs = {
    'depth': 11,
    'iterations':5000,
    'random_seed':0,
    'learning_rate':0.02,
    'od_type':'Iter',
    'od_wait':300,
    'task_type':'GPU',
                   }
model = CatBoostRegressor(**regressor_kwargs)

In [4]:
model.load_model(TRAINED_MODEL_FPATH)
print(model.get_params())
print(model.random_seed_)
print(model.learning_rate_)

{'task_type': 'GPU', 'random_seed': 0, 'od_type': 'Iter', 'od_wait': 300, 'loss_function': 'RMSE', 'depth': 11, 'learning_rate': 0.02, 'iterations': 3995, 'verbose': 100, 'logging_level': 'Verbose'}
0
0.01999999955


In [8]:
from catboost import Pool

test_X_df = pd.read_hdf(DATA_FPATH, 'test_X')
sample_submissions_df = pd.read_csv(SAMPLE_SUBMISSION_FPATH, index_col=0)

orig_test_X_df = test_X_df.copy()


assert test_X_df[orig_test_X_df.columns].equals(orig_test_X_df)
del orig_test_X_df

float64_cols = test_X_df.dtypes[test_X_df.dtypes == np.float64].index.tolist()

test_X_df[float64_cols] = test_X_df[float64_cols].astype(np.float32)
test_X_df = test_X_df.loc[sample_submissions_df.index,:]
test_prediction = model.predict(Pool(data=test_X_df))
test_prediction[test_prediction > 20] = 20
test_prediction[test_prediction < 0]  = 0
sample_submissions_df['item_cnt_month'] = test_prediction

In [9]:
sample_submissions_df.to_csv('submission.csv')

In [10]:
sample_submissions_df.head()

,item_cnt_month
ID,
0,0.647621
1,0.546208
2,0.879024
3,0.550725
4,3.763358
